In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

from tensorflow import keras

import os
import warnings
warnings.filterwarnings("ignore")
os.environ['GLOG_minloglevel'] = '2' 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2024-06-04 21:16:52.199831: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 21:16:52.250782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 21:16:53.568842: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Función para dibujar los puntos de referencia de la mano y las conexiones entre ellos
def draw_hand_landmarks(frame, landmarks, COLOR, margin=30):
    height, width, _ = frame.shape  # Obtiene las dimensiones del frame
    x_min, y_min, x_max, y_max = width, height, 0, 0  # Inicializa los límites del rectángulo que rodea la mano

    for i, point in enumerate(landmarks.landmark):
        x, y = int(point.x * width), int(point.y * height)  # Calcula las coordenadas del punto en píxeles
        color = (0, 255, 0) if i % 4 == 0 else (0, 0, 255)  # Alterna el color entre verde y rojo para cada punto
        cv2.circle(frame, (x, y), int(height * 0.01), color, -1)  # Dibuja un círculo en el punto

        # Actualiza las coordenadas mínimas y máximas para dibujar un rectángulo alrededor de la mano
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x)
        y_max = max(y_max, y)

    # Definir conexiones solo para el contorno de la mano y los dedos
    connections = [[1, 2], [2, 3], [3, 4],  # Dedo pulgar
                   [5, 6], [6, 7], [7, 8],  # Dedo índice
                   [9, 10], [10, 11], [11, 12],  # Dedo medio
                   [13, 14], [14, 15], [15, 16],  # Dedo anular
                   [17, 18], [18, 19], [19, 20],  # Dedo meñique
                   [0, 1],[0, 17],[1, 5], [5, 9], [9, 13], [13, 17]]  # Conexiones de la palma de la mano

    # Dibujar las conexiones entre los puntos de referencia de la mano
    for connection in connections:
        cv2.line(frame, (int(landmarks.landmark[connection[0]].x * width), int(landmarks.landmark[connection[0]].y * height)),
                         (int(landmarks.landmark[connection[1]].x * width), int(landmarks.landmark[connection[1]].y * height)), COLOR, 2)

    hand_width = x_max - x_min
    hand_height = y_max - y_min

    # Calculate the side length of the square bounding box
    square_side = max(hand_width, hand_height)

    # Calculate the coordinates of the square bounding box
    x_min_square = x_min + hand_width // 2 - square_side // 2
    x_max_square = x_min_square + square_side
    y_min_square = y_min + hand_height // 2 - square_side // 2
    y_max_square = y_min_square + square_side

    # Añadir un margen al cuadrado
    x_min_square = max(0, x_min_square)
    y_min_square = max(0, y_min_square)
    x_max_square = min(width, x_max_square)
    y_max_square = min(height, y_max_square)

    # Dibujar el cuadrado que rodea la mano
    cv2.rectangle(frame, (x_min_square-margin, y_min_square-margin), (x_max_square+margin, y_max_square+margin), COLOR, 2)

    # Return the square bounding box instead of the hand rectangle
    return frame, (x_min_square-margin, y_min_square-margin, x_max_square+margin, y_max_square+margin)

# Función para mapear las etiquetas
def getLetter(result):
    classlabels = {'A': 0,
                    'B': 1,
                    'C': 2,
                    'D': 3,
                    'E': 4,
                    'F': 5,
                    'G': 6,
                    'I': 7,
                    'K': 8,
                    'L': 9,
                    'M': 10,
                    'N': 11,
                    'O': 12,
                    'P': 13,
                    'Q': 14,
                    'R': 15,
                    'S': 16,
                    'T': 17,
                    'U': 18}
    
    try:
        return classlabels[int(result)]
    except Exception as e:
        # retornar la letra correspondiente al resultado
        return list(classlabels.keys())[list(classlabels.values()).index(result)]

# Función para capturar frames de la cámara y guardarlos en carpetas correspondientes a cada letra
def capture_frames(hands, model, path):

    cap = cv2.VideoCapture(0)  # Inicializa la cámara
    start_time = time.time()  # Inicializa el tiempo de captura
    frame_count = 0  # Inicializa el contador de frames

    texto = ''

    while True:
        ret, frame = cap.read()  # Lee un frame de la cámara
        letter = ''

        # Si no se pudo leer el frame, termina el ciclo
        if not ret:
            print("Error al leer el frame")
            break

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convierte el frame de BGR a RGB

        hands_results = hands.process(img)  # Procesa el frame con el modelo de manos

        drawn_frame = frame.copy()  # Crea una copia del frame

        # Dibuja puntos y líneas para las manos en la imagen negra
        if hands_results.multi_hand_landmarks:
            for landmarks in hands_results.multi_hand_landmarks:
                drawn_frame, hand_rect = draw_hand_landmarks(frame, landmarks, (0, 0, 255))  # Dibuja puntos y líneas para las manos

                # recortar la mano y guardarla 
                try:
                    reshaped_frame = drawn_frame[hand_rect[1]:hand_rect[3], hand_rect[0]:hand_rect[2]] # Recorta el cuadrado que rodea la mano y lo redimensiona
                    #reshaped_frame = cv2.flip(reshaped_frame, 1)  # Voltea la imagen horizontalmente
                    reshaped_frame = cv2.cvtColor(reshaped_frame, cv2.COLOR_BGR2GRAY)  # Convierte la imagen a escala de grises
                    reshaped_frame = cv2.resize(reshaped_frame, (28, 28), interpolation=cv2.INTER_AREA) # Redimensiona la imagen a 28x28 píxel  es

                    reshaped_frame = reshaped_frame.reshape(1, 28, 28,  1)

                    result = np.argmax(model.predict(reshaped_frame, verbose=0), axis=-1)[0]  # Clasifica la imagen
                    letter = getLetter(result)
                    
                    if letter == 'Q':
                        cv2.putText(frame, f'Borrar', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                        if (frame_count % 60 == 0):
                            texto = texto[:-1]

                    elif letter == 'O':
                        cv2.putText(frame, f'Espacio', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                        if (frame_count % 60 == 0):
                            texto += ' '

                    elif letter == 'B':
                        cv2.putText(frame, f'Guardar', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                        if (texto != '') and (frame_count % 60 == 0):
                            with open(f'{path}texto.txt', 'a') as f:
                                f.write(texto + '\n')
                    else:
                        cv2.putText(frame, f'{letter}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                        if frame_count % 60 == 0:
                            texto += letter
                    frame_count += 1  # Incrementa el contador de frames
                    
                except Exception as e:
                    cv2.putText(frame, f'Error: {e}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    pass

        elapsed_time = int(time.time() - start_time)  # Calcula el tiempo transcurrido
        cv2.putText(frame, f'Tiempo: {elapsed_time} seg', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.putText(frame, f'Texto: {texto}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.imshow('Pose Detection', frame)  # Muestra el frame con las manos detectadas
        # Si se presiona la tecla 'q', termina el ciclo
        if (cv2.waitKey(1) & 0xFF == ord('q')): # or letter == 'B':
            break

    cap.release()  # Libera la cámara
    cv2.destroyAllWindows() # Cerrar todas las ventanas
    hands.close() # Cerrar el modelo de manos




In [6]:
model = keras.models.load_model('./models/model3.h5')
path = 'out/' # 
mp_hands = mp.solutions.hands # Importar el modelo de manos de MediaPipe
hands = mp_hands.Hands(max_num_hands=1) # Inicializar el modelo de manos

capture_frames(hands, model, path) # Capturar frames cada 15 frames y guardar 100 frames por letra en la carpeta "classes_poses_cut"


I0000 00:00:1717556286.365088   16138 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1717556286.365694   54230 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.8-manjaro1.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
